In [1]:
from Marc_to_Bibframe.Work.work import Work
from Marc_to_Bibframe.Marc.marcWork import MarcWork
import xml.etree.ElementTree as etree
from xml.dom.minidom import parse
from pyfuseki import FusekiUpdate
from rdflib import Graph, Namespace, URIRef
from rdflib.plugins.stores import sparqlstore
from rdflib.namespace import RDF, RDFS

from Marc_to_Bibframe.Work.workAdmin import WorkAdmin
from Marc_to_Bibframe.Work.language import Language
from Marc_to_Bibframe.Work.audience import Audience
from Marc_to_Bibframe.Work.classification import Classification
from Marc_to_Bibframe.Work.chamada import Chamada
from Marc_to_Bibframe.Work.primaryContribution import PrimaryContribution
from Marc_to_Bibframe.Work.contribution import Contributions
from Marc_to_Bibframe.Work.title import Title
from Marc_to_Bibframe.Work.subject import Subject
from Marc_to_Bibframe.Work.type import Type
from Marc_to_Bibframe.Work.work import Work


In [2]:
path_marc = r'Koha/E1P1.xml'
marc_file = parse(path_marc)
records = marc_file.getElementsByTagName('record')
record = records[0]
marcxml = etree.fromstring(record.toxml())
workMarc = MarcWork(marcxml)

In [ ]:
count = 1
uri = f"http://bibliokeia.com/bibframe/work"
BFwork = URIRef(f"http://bibliokeia.com/bibframe/work/{count}") 
BFinstance = URIRef(f"http://bibliokeia.com/bibframe/instance/{count}")
g = Graph(identifier=BFwork)
#Prefix
g.bind('rdf', RDF)
g.bind('rdfs', RDFS)
BF = Namespace("http://id.loc.gov/ontologies/bibframe/")
g.bind('bf', BF)
BFLC = Namespace("http://id.loc.gov/ontologies/bflc/")
g.bind('bflc', BFLC)
MADSRDF = Namespace("http://www.loc.gov/mads/rdf/v1#")
g.bind('madsrdf', MADSRDF)
# BK = Namespace("http://bibliokeia.com/bibframe/work/")
# g.bind('bk', BK)

BFwork = URIRef(f"http://bibliokeia.com/bibframe/work/{count}") 
g.add((BFwork, RDF.type, BF.Work))
g.add((BFwork, RDF.type, BF.Text))

g = WorkAdmin(g, BFwork, count, workMarc, BF)
g = Language(g, BFwork, workMarc, BF)
if workMarc.Audience():
    g = Audience(g, BFwork, workMarc, BF)
g = Classification(g, BFwork, workMarc, BF)
g = Chamada(g, BFwork, workMarc, BF)
g = PrimaryContribution(g, BFwork, workMarc, count, BF, BFLC)
if workMarc.Contributions():
    g = Contributions(g, BFwork, workMarc, count, uri, BF, BFLC)
g = Title(g, BFwork, workMarc, BF)
g = Subject(g, BFwork, uri, count, workMarc, BF, MADSRDF, RDFS)
g = Type(g, BFwork, workMarc, BF)
g.add((BFwork, BF.hasInstance, BFinstance))
nt = g.serialize(format='nt')
print(nt)

In [ ]:
count = 2
path_marc = r'Koha/E1P1.xml'
marc_file = parse(path_marc)
records = marc_file.getElementsByTagName('record')
record = records[1]
marcxml = etree.fromstring(record.toxml())
workMarc = MarcWork(marcxml)
BFwork = URIRef(f"http://bibliokeia.com/bibframe/work/{count}") 
BFinstance = URIRef(f"http://bibliokeia.com/bibframe/instance/{count}")
g = Work(count, workMarc, BFwork, BFinstance)
nt = g.serialize(format='nt')
print(nt)

In [18]:
def Make_Graph(nt):
    G1 = """
    PREFIX bk: <http://bibliokeia.com/bibframe/work/>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/> 
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX bflc: <http://id.loc.gov/ontologies/bflc/>
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

    INSERT DATA {
        GRAPH bk:"""

    G2 = " {"

    G3 = """}
    }"""

    G = G1+str(count)+" { \n"+nt+"}}"

    return G

In [21]:
update_endpoint = 'http://localhost:3030/teste2/update'
query_endpoint = 'http://localhost:3030/teste2/query'
store = sparqlstore.SPARQLUpdateStore()
store.open((query_endpoint, update_endpoint))


count = 2
for record in records[2:]:
    count += 1
    marcxml = etree.fromstring(record.toxml())
    workMarc = MarcWork(marcxml)
    print(workMarc.Title().get('title'))
    BFwork = URIRef(f"http://bibliokeia.com/bibframe/work/{count}") 
    BFinstance = URIRef(f"http://bibliokeia.com/bibframe/instance/{count}")
    g = Work(count, workMarc, BFwork, BFinstance)
    nt = g.serialize(format='nt')
    G = Make_Graph(nt)
    store.update(G)


Cooperação Brasil-França
<I=     1> Bienal de Pesquisa da Fundação Oswaldo Cruz, 7 a 11 de dezembro de 1998.
Contribuição da pós-graduação brasileira para o desenvolvimento sustentável
FAPESP
Para uma história da FAPESP
Relatório de atividades 2003-2008
V Jornada científica de pos-graduação da FIOCRUZ
Anais
Anais
Relatório de atividades 2007
Resumos
Resumos
Resumos
Resumos
Resumos
Relatório de atividades 2010


In [10]:
update_endpoint = 'http://localhost:3030/teste2/update'
query_endpoint = 'http://localhost:3030/teste2/query'
store = sparqlstore.SPARQLUpdateStore()
store.open((query_endpoint, update_endpoint))

store.update(G)

In [5]:
D = """
PREFIX bk: <http://bibliokeia.com/bibframe/work/>

DELETE { GRAPH bk:1 { ?s ?p ?o } } WHERE { GRAPH bk:1 { ?s ?p ?o } }
"""

update_endpoint = 'http://localhost:3030/teste2/update'
query_endpoint = 'http://localhost:3030/teste2/query'
store = sparqlstore.SPARQLUpdateStore()
store.open((query_endpoint, update_endpoint))

store.update(D)

In [5]:
update_endpoint = 'http://localhost:3030/teste2/update'
query_endpoint = 'http://localhost:3030/teste2/query'
store = sparqlstore.SPARQLUpdateStore()
store.open((query_endpoint, update_endpoint))

store.update(D)

In [8]:
g.serialize(format='nt')

'<http://bibliokeia.com/bibframe/work/1> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://id.loc.gov/ontologies/bibframe/Work> .\n'